In [1]:
# Importing packages and loading env:
import pandas as pd
import numpy as np
import re
import json
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
load_dotenv()
from src.api import exchangerate_api_request
from src.api import battuta_request_authorized
from src.api import foursquare_request_venues_authorized
from src.api import foursquare_menu_hours_authorized
from src.api import foursquare_get_id_authorized
from src.webscraping import get_soup
from src.clean import resub_list


df_final = pd.read_csv('./input/cleaned_enriched_df.csv')

In [8]:
df_final.head()

,name,state,region,city,latitude,longitude,cuisine,stars,min_price_EUR,max_price_EUR,url,foursquare_id,foursquare_name,foursquare_address
0,Kilian Stuba,Austria,Austria,Kleinwalsertal,47.348580,10.17114,Creative,1,90,120,https://guide.michelin.com/at/en/vorarlberg/kl...,50e86b67e4b097a7b607168a,Kilian Stuba,Österreich
1,Pfefferschiff,Austria,Austria,Hallwang,47.837870,13.07917,Classic cuisine,1,59,120,https://guide.michelin.com/at/en/salzburg-regi...,501ec7bfe4b0b87f23ea6dd6,Pfefferschiff,Salzburg
2,Esszimmer,Austria,Austria,Salzburg,47.806850,13.03409,Creative,1,45,128,https://guide.michelin.com/at/en/salzburg-regi...,4cec14ff678aa093081afcea,Esszimmer,Müllner Hauptstr. 33
3,Carpe Diem,Austria,Austria,Salzburg,47.800010,13.04006,Market cuisine,1,61,118,https://guide.michelin.com/at/en/salzburg-regi...,4bc48bca461576b03f088032,Carpe Diem Finest Fingerfood,Getreidegasse 50
4,Edvard,Austria,Austria,Wien,48.216503,16.36852,Modern cuisine,1,33,158,https://guide.michelin.com/at/en/vienna/wien/r...,51bcab0d498e39a39ac36f7a,Edvard Restaurant,Schottenring 24


In [ ]:
# Web scraping https://guide.michelin.com to get services:
# names_list = []
i = 0
services_list = []
for e in df_final['url']:
    soup = get_soup(e)
    services = soup.select('.restaurant-details__services--content')
    services_rest = [re.sub('\n','',e.text) for e in services]
    services_list.append([df_final.at[i,'name'], services_rest])
    i+=1
print(services_list)

In [ ]:
# % de similitud entre dos string
for i in range(len(restaurants_name)):
    print([restaurants_name[i], df_final.at[i,'name']])

In [ ]:
# foursquare_request_venues_authorized('explore', df_final['latitude'][0], df_final['longitude'][0], 'restaurant')

In [ ]:
# foursquare_menu_hours_authorized('hours',data['id'])

In [ ]:
# df_final

In [10]:
# df_final.to_csv('./input/cleaned_enriched_df.csv', index=False)